In [ ]:
# Purpose: Setup imports and dataset paths.
# =========================
# SCUT-FBP5500 Beauty Score
# =========================
from fastai.vision.all import *
from pathlib import Path

# -------------------------
# Paths
# -------------------------
data_path = Path("~/Downloads/SCUT-FBP5500_v2") # change to your own path
images_path = data_path / "Images"
train_txt = data_path / "train_test_files/split_of_60%training and 40%testing/train.txt"
valid_txt = data_path / "train_test_files/split_of_60%training and 40%testing/test.txt"

In [ ]:
# Purpose: Read split files into DataFrames and prepare combined df.
# -------------------------
# Read splits into DataFrames
# -------------------------
import pandas as pd

train_df = pd.read_csv(train_txt, sep=' ', header=None, names=['filename', 'rating'])
valid_df = pd.read_csv(valid_txt, sep=' ', header=None, names=['filename', 'rating'])

# Remove .jpg extension
train_df['filename'] = train_df['filename'].str.replace('.jpg', '', regex=False)
valid_df['filename'] = valid_df['filename'].str.replace('.jpg', '', regex=False)

# Add split column
train_df['is_valid'] = False
valid_df['is_valid'] = True

# Combine
df = pd.concat([train_df, valid_df], ignore_index=True)

'/content'

In [ ]:
# Purpose: Build the DataBlock and create dataloaders.
set_seed(42, reproducible=True)

# -------------------------
# DataBlock with ColReader
# -------------------------
dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_x=ColReader('filename', pref=images_path, suff='.jpg'),
    get_y=ColReader('rating'),
    splitter=ColSplitter('is_valid'),
    item_tfms=Resize(224),
    batch_tfms=aug_transforms(do_flip=False)
)

dls = dblock.dataloaders(df, bs=32)


In [ ]:
# Purpose: Visualize a batch and create the ResNet34 regression learner.
dls.show_batch()

# -------------------------
# Learner
# -------------------------
learn = vision_learner(dls, resnet34, loss_func= MSELossFlat(), metrics = rmse)

In [ ]:
learn.lr_find()

In [ ]:
from fastai.callback.tracker import SaveModelCallback, EarlyStoppingCallback

# -------------------------
# Train with callbacks
# -------------------------
learn.fine_tune(
    15,              # total epochs
    base_lr=0.0017,  # suggested LR
    freeze_epochs=3, # first 3 epochs train only the head
    cbs=[
        SaveModelCallback(monitor='valid_loss', fname='best_beauty_model'),  # saves best model
        EarlyStoppingCallback(monitor='valid_loss', patience=3, min_delta=0.01) # stop if improvement < 0.01 for 3 epochs
    ]
)

SaveModelCallback(monitor='valid_loss', fname='best_beauty_model')


In [ ]:
from fastai.learner import load_learner
import warnings
import torch

warnings.filterwarnings('ignore')

# Load the model
learner_tuned = load_learner('/root/beauty_model.pkl', cpu=False)

print(f"Using device: {learner_tuned.dls.device}")
print(f"Processing {len(df_all_sorted)} images...")

# Batch prediction on GPU (MUCH faster!)
test_dl = learner_tuned.dls.test_dl(df_all_sorted['image_path'].tolist())
preds, _ = learner_tuned.get_preds(dl=test_dl)

# Extract scores
df_all_sorted['beauty_score'] = preds.squeeze().cpu().numpy()

print("Done!")